# Web-scraping: сбор данных из баз данных и интернет-источников


## Извлечение кода JavaScript из HTML и обработка JSON-строк

*Алла Тамбовцева*

### Очень краткое введение в регулярные выражения

Регулярные выражения – выражения, последовательности символов, которые позволяют искать совпадения в тексте. Выражаясь более формально, они помогают найти подстроки определенного вида в строке. Еще о регулярных выражениях можно думать как о шаблонах, в которые мы можем подставлять текст, и этот текст либо соответствует шаблону, либо нет. 

В самом простом случае в качестве регулярного выражения может использоваться обычная строка. Например, чтобы найти в предложении *Кошка сидит под столом.* слово *Кошка*, ничего специального применять не нужно, достаточно воспользоваться оператором `in`. Если нас интересует слово *кошка* в любом регистре, то это уже более интересная задача. Правда, ее все еще можно решить без регулярных выражений, приведя все слова в строке к нижнему регистру. А что, если у нас будет текст подлиннее, и в нем необходимо «обнаружить» *кошку* в разных падежах? И еще производные слова вроде *кошечка*? Тут уже удобнее написать некоторый шаблон, чтобы не создавать длинный список слов с разными формами слова *кошка*. Давайте немного потренируемся (но не на кошках).

Импортируем модуль `re` для работы с регулярными выражениями:

In [1]:
import re

В качестве игрушечного примера возьмем обычную строку со странным текстом (текст невнятный, но отражает эволюцию смеха на пути к сессии):

In [2]:
data0 = "ha haha ha-ha hah heh. hse."

Найдем в этой строке все подстроки, которые соответствуют шаблону `h.h` – вместо точки может быть любой символ (буква, цифра, пробел и прочие знаки). Воспользуемся функцией `findall()`, она возвращает список совпадений:

In [3]:
re.findall("h.h", data0)

['hah', 'hah', 'heh']

Если нужны именно точки, символ `.` нужно экранировать с помощью `\`, в такой записи слэш показывает, что мы ищем именно точку, а не используем ее как специальный символ, принятый в синтаксисе регулярных выражений. Итак, найдем все «слова», начинающиеся с `h`, состоящие из четырех символов, последний из которых – точка:

In [4]:
re.findall("h..\.", data0)

['heh.', 'hse.']

Точка – далеко не единственный специальный символ в регулярных выражениях. Так, символ `+`  показывает, что нас интересуют случаи, когда элемент, стоящий слева от `+`, встречается не менее одного раза. Найдем подстроки, где точно есть буква `h`, а за ней стоит хотя бы одна буква `a`:

In [5]:
# подстроки с h с хотя бы с одной буквой a
re.findall("ha+", data0)

['ha', 'ha', 'ha', 'ha', 'ha', 'ha']

Если мы допускаем, что буквы `a` может не быть совсем, нам понадобится другой символ – символ `*` (ноль и более вхождений элемента, стоящего слева от `*`):

In [6]:
# подстроки, где точно есть h, а буква a встречается или нет
re.findall("ha*", data0)

['ha', 'ha', 'ha', 'ha', 'ha', 'ha', 'h', 'h', 'h', 'h']

А если нас интересуют случаи, когда какой-то символ встречается ноль раз или один раз, то пригодится символ `?`:

In [7]:
# подстроки haha или ha-ha, с дефисом посередине и без него
re.findall("ha-?ha", data0)

['haha', 'ha-ha']

Особую роль в регулярных выражениях играют скобки разного вида. Круглые скобки могут использоваться для объединения символов в группы, а квадратные – для перечисления всех вариантов, которые могут встретиться в некотором месте строки:

In [8]:
# hah или heh с точкой или пробелом на конце
# \s – обозначение пробела (от space)

re.findall("h[ae]h[\.\s]", data0)

['hah ', 'heh.']

В квадратные скобки также можно вписывать последовательности – готовые перечни известных символов:

* `[a-z]`: строчные буквы английского алфавита;
* `[A-Z]`: заглавные буквы английского алфавита;
* `[а-я]`: строчные буквы русского алфавита;
* `[А-Я]`: заглавные буквы русского алфавита;
* `[0-9]`: цифры от 0 до 9.

Проверим, есть ли в нашей строке цифры:

In [9]:
re.findall("[0-9]", data0)  # нет, мы и не ждали

[]

А теперь проверим, есть ли в нашей строке последовательности ровно из трех строчных английских букв. Для этого пригодится еще один вид скобок – фигурные. В фигурных скобках указывают количество символов, которое необходимо найти:

In [10]:
# последовательности ровно из трех английских букв
re.findall("[a-z]{3}", data0)

['hah', 'hah', 'heh', 'hse']

Если мы не знаем точное количество символов, но знаем интервал, его границы тоже можно указать в фигурных скобках через запятую:

In [11]:
# последовательности из 3-4 английских букв
re.findall("[a-z]{3,4}", data0)

['haha', 'hah', 'heh', 'hse']

Границы интервала можно опускать:

In [12]:
# последовательности не менее, чем из 3 английских букв
re.findall("[a-z]{3,}", data0)

['haha', 'hah', 'heh', 'hse']

In [13]:
# последовательности не более, чем из 3 английских букв (пустые тоже есть)
re.findall("[a-z]{,3}", data0)

['ha',
 '',
 'hah',
 'a',
 '',
 'ha',
 '',
 'ha',
 '',
 'hah',
 '',
 'heh',
 '',
 '',
 'hse',
 '',
 '']

Давайте повнимательнее посмотрим на поиск цифр и чисел, может пригодиться, например, для обработки номеров телефонов или адресов. Создадим другую, более вразумительную строку:

In [14]:
data1 = "+7(906)000-11-23 Alla A" 

Пока просто найдем все цифры:

In [15]:
re.findall("[0-9]", data1)

['7', '9', '0', '6', '0', '0', '0', '1', '1', '2', '3']

Для поиска цифр вместо последовательности часто используют ее сокращенную версию – специальный символ `\d` (от *digits*, экранируется с помощью слэша, чтобы не путать с обычной буквой *d*):

In [16]:
re.findall("\d", data1)

['7', '9', '0', '6', '0', '0', '0', '1', '1', '2', '3']

Цифры нашли, но ведь цифры в строке – далеко не всегда номер телефона, теоретически они могут быть и в адресе (как обычном, так и электронном), и в названии сайта.  Напишем паттерн для поиска именно номера телефона в предположении, что:

* телефон точно начинается с `+7`;
* после `+7` обязательно стоят скобки вокруг первых трех цифр;
* а вот дефисы между группами цифр могут отсутствовать):

In [17]:
# \+7: экранируем +, чтобы не путать со специальным символом +
# (\d{3}\): набор из 3 цифр в скобках
# \d{3}: набор из 3 цифр
# -?: дефис встречается 0 или 1 раз
# \d{2}: набор из 2 цифр

re.findall("\+7\(\d{3}\)\d{3}-?\d{2}-?\d{2}", data1)

['+7(906)000-11-23']

Если допустить, что телефон может начинаться с `8`, а не только с `+7`, выражение будет выглядеть так:

In [18]:
# \+?: + встречается 0 или 1 раз
# после 7 или 8

re.findall("\+?[78]\(\d{3}\)\d{3}-?\d{2}-?\d{2}", data1)

['+7(906)000-11-23']

Проверим на другой строке:

In [19]:
data2 = "+7(906)000-11-23 Alla Borisovna 8(906)111-00-23 Alla Andreevna" 
re.findall("\+?[78]\(\d{3}\)\d{3}-?\d{2}-?\d{2}", data2)

['+7(906)000-11-23', '8(906)111-00-23']

Ну, а если допустить, что «приставки» `+7` или `8` может вообще не быть, то понадобится еще один `?`:

In [20]:
data3 = "+7(906)000-11-23 Alla Borisovna 8(906)111-00-23 Alla Andreevna (999)233-00-21 Alla" 
re.findall("\+?[78]?\(\d{3}\)\d{3}-?\d{2}-?\d{2}", data3)

['+7(906)000-11-23', '8(906)111-00-23', '(999)233-00-21']

Итак, на этом краткое введение в регулярные выражения мы закончим, сейчас увидим, зачем они могут понадобиться при парсинге, даже если мы выгружаем информацию с помощью BeautifulSoup.

### Извлечение информации из кода JavaScript внутри HTML

Совсем недавно у нас было домашнее задание на парсинг страницы фильма «Не покидай...» с сайта www.kino-teatr.ru. Сайт некоммерческий, довольно дружелюбный, позволяет свободно выгружать информацию (но уже не всегда!). 

У него есть одна особенность: число лайков и дизлайков, поставленных актерам пользователями, загружается на страницу динамически, то есть автоматически «подтягивается» с сервера при загрузке страницы в определенный момент времени. На практике это выливается в то, что найти нужную информацию по тэгам просто невозможно, ее нет в основном коде HTML. Как быть? Понять, как выглядит запрос данных, который отправляется на сервер, и выяснить, где хранятся нужные нам данные. Мы рассмотрим несложный случай, когда сайт забирает информацию из строки JSON, которая находится на странице, но внутри кода, написанного на JavaScript. Такое можно встретить на страницах с результатами каких-нибудь игр или на сайтах, посвященных динамике цен или курсу валют (другой вопрос, что не всегда JSON прямо так явно находится в том же файле, где и код HTML).

Так как ранее мы обсуждали довольно базовый парсинг, в домашнем задании не требовалось собирать число лайков и дизлайков, эти значения были даны в виде готового списка словарей. Обновим задачу – теперь нам нужно собрать только id актеров и информацию о числе голосов за и против. 

Начало работы стандартное – загружаем код HTML страницы по ссылке и преобразуем его в объект BeautifulSoup:

In [21]:
import requests
from bs4 import BeautifulSoup

In [22]:
page = requests.get("https://www.kino-teatr.ru/kino/movie/sov/4319/titr/")
soup = BeautifulSoup(page.text)

Просто найти на странице кнопки красного и зеленого цвета и забрать с них текст не получится:

![](НП01.jpeg)

Поэтому для этого на нужно найти код JavaScript, где есть записи с числами `plus` и `minus` с привязкой к id актеров. Код JavaScript, если он не вынесен в отдельный файл, заключается в тэги `<script>`. Если мы внимательно изучим исходный код страницы, мы заметим, что нужный нам блок кода имеет атрибут `type` со значением `text/javascript`, и среди таких блоков он находится на 7-ом месте:

In [23]:
# выполняем поиск и извлекаем текст – код JavaScript в виде строки

text = str(soup.find_all("script", {"type" : "text/javascript"})[6])
text

'<script type="text/javascript">\r\n<!--\r\n\r\n// Рисуем кнопки в javascript\r\nfunction print_role_rating_buttons(role)\r\n{\r\n    var span = $("#role_rating_"+role.id);\r\n    var site = \'desktop\'\r\n\r\n    if (span) {\r\n        if (site == \'desktop\') {\r\n            span.html("<input type=\'button\' value=\'+ " + role.plus +"\' class=\'acter_vote acter_vote_plus rating_button like\' title=\'Мне понравилась эта актёрская работа\' mark=\'plus\' clicked=\'0\' role_id=\'" + role.id +"\' onClick=\'return sendVote(this)\' mark=\'plus\'> <input type=\'button\' value=\'- " + role.minus + "\' class=\'acter_vote acter_vote_minus rating_button dislike\' title=\'Мне не понравилась эта актёрская работа\' mark=\'minus\' clicked=\'0\' role_id=\'" + role.id +"\' onClick=\'return sendVote(this)\' mark=\'minus\'>")\r\n        } else {\r\n            span.html("<button class=\'acter_vote acter_vote_plus ui-btn ui-btn-inline ui-corner-all ui-btn-text-left ui-btn-icon-left ui-icon-plus\' title=

Блок с кодом довольно большой, в нем содержатся разные функции для отрисовки и обновления информации на кнопках (на них отображаются голоса за и против, на них же можно кликать после авторизации, чтобы записать свой голос). Нам же понадобится часть кода после функции `print_role_rating_buttons ()`, так как именно под ней располагается необходимая информация. Воспользуемся методом `.split()` и разобьем строку на части:

In [24]:
for_votes = text.split("function print_all_role_rating_buttons ()")[1]
print(for_votes)


{
    print_role_rating_buttons({ id:"160325", plus:"55", minus:"2", voted:"" });
print_role_rating_buttons({ id:"1928641", plus:"28", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1966145", plus:"23", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973000", plus:"23", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973001", plus:"24", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973002", plus:"25", minus:"0", voted:"" });
print_role_rating_buttons({ id:"1973003", plus:"25", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973005", plus:"25", minus:"2", voted:"" });
print_role_rating_buttons({ id:"1973007", plus:"28", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973008", plus:"22", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973010", plus:"34", minus:"6", voted:"" });
print_role_rating_buttons({ id:"1973011", plus:"46", minus:"3", voted:"" });
print_role_rating_buttons({ id:"2088754", plus:"29", minus:"2", voted:

Хотя с синтаксисом JavaScript знакомы немногие, не очень сложно догадаться, что означают записи такого вида:

    print_role_rating_buttons({ id:"16800", plus:"156", minus:"4", voted:"" });
    
Такая строка кода активирует функцию `print_role_rating_buttons()` – применяет ее к такому набору данных и наносит на кнопки, соответствующие актеру с id 16800 значения 156 (зеленая кнопка) и 4 (красная кнопка).

*Примечание 1*. Синхронизация не происходит мгновенно, актуальное значение 156 подставляется на кнопку не сразу, там вполне может стоять значение 155, а мы, благодаря значениям из JavaScript, обладаем более актуальной информацией. Это стоит иметь в виду при выгрузке информации, когда мы сверяем полученные результаты и то, что видим на экране.

*Примечание 2*. Значение `voted` в коде выше является непустым, в случае, если пользователь зарегистрирован и оценивал игру актера (по крайней мере, какое-то время после голосования значение в `voted` держится, потом на самой странице отметка о голосовании остается, а в JSON стирается). Сравним скрин и фрагменты кода для такого случая (да, ради иллюстрации и проверки кода JavaScript я зарегистрировалась):

![](НП02.jpeg)
    
Код:

    print_role_rating_buttons({ id:"56008", plus:"102", minus:"10", voted:"" });
    print_role_rating_buttons({ id:"132139", plus:"116", minus:"5", voted:"" });


Все фрагменты кода выше – это обычные строки, объекты типа *string*. Поэтому с помощью регулярных выражений мы сможем найти в них записи в фигурных скобках, затем сделать из них словари, а из списка словарей с единообразными ключами собрать датафрейм!

Найдем все подстроки, соответствующие шаблону `\{.+\}`, то есть просто все наборы символов, заключенных в фигурные скобки (скобки экранируем, чтобы не перепутать со специальными скобками в регулярных выражениях):

In [25]:
votes_str = re.findall("\{.+\}", for_votes)
votes_str

['{ id:"160325", plus:"55", minus:"2", voted:"" }',
 '{ id:"1928641", plus:"28", minus:"1", voted:"" }',
 '{ id:"1966145", plus:"23", minus:"1", voted:"" }',
 '{ id:"1973000", plus:"23", minus:"1", voted:"" }',
 '{ id:"1973001", plus:"24", minus:"1", voted:"" }',
 '{ id:"1973002", plus:"25", minus:"0", voted:"" }',
 '{ id:"1973003", plus:"25", minus:"1", voted:"" }',
 '{ id:"1973005", plus:"25", minus:"2", voted:"" }',
 '{ id:"1973007", plus:"28", minus:"1", voted:"" }',
 '{ id:"1973008", plus:"22", minus:"1", voted:"" }',
 '{ id:"1973010", plus:"34", minus:"6", voted:"" }',
 '{ id:"1973011", plus:"46", minus:"3", voted:"" }',
 '{ id:"2088754", plus:"29", minus:"2", voted:"" }',
 '{ id:"16800", plus:"156", minus:"4", voted:"" }',
 '{ id:"16801", plus:"109", minus:"26", voted:"" }',
 '{ id:"16803", plus:"125", minus:"12", voted:"" }',
 '{ id:"16802", plus:"146", minus:"8", voted:"" }',
 '{ id:"16804", plus:"140", minus:"6", voted:"" }',
 '{ id:"56008", plus:"102", minus:"10", voted:"" }

Выберем один элемент списка и изучим его:

In [26]:
v = votes_str[0]
v

'{ id:"160325", plus:"55", minus:"2", voted:"" }'

На первый взгляд, этот элемент представляет собой полноценный словарь, просто заключенный в кавычки. 
Однако есть проблема: чтобы переделать строку в словарь, нужна строка, соответствующая формату JSON, а здесь не хватает кавычек вокруг ключей. Как эту проблему решить? Найти по какому-то паттерну эти ключи и доклеить вокруг них кавычки с помощью функции `sub()` из модуля `re` для «умной» замены (не обычная замена, так как мы не конкретный набор символов заменяем на другой, а ищем совпадения по некоторому общему шаблону и их изменяем). 

Поиск будет простой – в строке с «неправильным» словарем содержатся либо цифры, либо буквы, либо пробелы с запятыми и знаками препинания, а нам нужны последовательности из одной и более букв:

In [27]:
re.findall("[a-z]+", v)

['id', 'plus', 'minus', 'voted']

Если были буквы были и строчные, и заглавные, код был бы таким – пример другой строки:

In [28]:
# | для или, один и более символов в обоих случаях

re.findall("[a-z]+|[A-Z]+", "{id : 23, FOR: 100, AGAINST: 45} ")

['id', 'FOR', 'AGAINST']

Напишем функцию `add_quotes()`, которая принимает на вход строку `x`, доклеивает к ней кавычки и убирает пробелы. Нам нужно преобразовать строку `x` в группу – объект из регулярных выражений – через `+` доклеить кавычки и на всякий случай убрать лишние пробелы:

In [29]:
# x.group(): функция sub() из re для умной замены
# работает не со строками, а с объектом специального типа – группа символов

def add_quotes(x):
    g = '"' + x.group() + '"'
    return g.strip()

Применяем функцию `sub` к одной строке `v` и проверяем:

In [30]:
# аргументы:
# шаблон, по которому ищем, что заменять
# функция, которая выполняет преобразование – доклеивает к результатм поиска кавычки
# строка, по который выполняем поиск

re.sub("[a-z]+", add_quotes, v)

'{ "id":"160325", "plus":"55", "minus":"2", "voted":"" }'

Все работает, применяем ко всем строкам и получаем список валидных json-строк:

In [31]:
votes_new = [re.sub("[a-z]+", add_quotes, v) for v in votes_str]
votes_new

['{ "id":"160325", "plus":"55", "minus":"2", "voted":"" }',
 '{ "id":"1928641", "plus":"28", "minus":"1", "voted":"" }',
 '{ "id":"1966145", "plus":"23", "minus":"1", "voted":"" }',
 '{ "id":"1973000", "plus":"23", "minus":"1", "voted":"" }',
 '{ "id":"1973001", "plus":"24", "minus":"1", "voted":"" }',
 '{ "id":"1973002", "plus":"25", "minus":"0", "voted":"" }',
 '{ "id":"1973003", "plus":"25", "minus":"1", "voted":"" }',
 '{ "id":"1973005", "plus":"25", "minus":"2", "voted":"" }',
 '{ "id":"1973007", "plus":"28", "minus":"1", "voted":"" }',
 '{ "id":"1973008", "plus":"22", "minus":"1", "voted":"" }',
 '{ "id":"1973010", "plus":"34", "minus":"6", "voted":"" }',
 '{ "id":"1973011", "plus":"46", "minus":"3", "voted":"" }',
 '{ "id":"2088754", "plus":"29", "minus":"2", "voted":"" }',
 '{ "id":"16800", "plus":"156", "minus":"4", "voted":"" }',
 '{ "id":"16801", "plus":"109", "minus":"26", "voted":"" }',
 '{ "id":"16803", "plus":"125", "minus":"12", "voted":"" }',
 '{ "id":"16802", "plus":"

Теперь нам осталось считать эти валидные JSON-строки с помощью Python – то есть превратить их в обычные питоновские словари.  Импортируем модуль `json`:

In [32]:
import json

Проверяем работу функции `loads()` на примере первой строки из полученного списка `votes_new`:

In [33]:
json.loads(votes_new[0])

{'id': '160325', 'plus': '55', 'minus': '2', 'voted': ''}

Напоминание: в модуле `json` есть две похожих функции:

* `load()` загружает данные из файла с расширением `.json`;
* `loads()` – загружает данные из JSON-строки.

Теперь применим эту функцию ко всем строкам в `votes_new` и сразу преобразуем получившийся список словарей в датафрейм:

In [34]:
import pandas as pd

In [35]:
ratings = pd.DataFrame([json.loads(v) for v in votes_new])
ratings

,id,plus,minus,voted
0,160325,55,2,
1,1928641,28,1,
2,1966145,23,1,
3,1973000,23,1,
4,1973001,24,1,
5,1973002,25,0,
6,1973003,25,1,
7,1973005,25,2,
8,1973007,28,1,
9,1973008,22,1,


Ура! Вся относительно тяжелая работа проделана, а собрать остальную информацию об актерах, чтобы затем склеить датафреймы по id, можно довольно стандартным образом.